In [1]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

2023-07-31 15:10:37 Didis-MacBook-Pro.local metapub.config[42116] WARNING NCBI_API_KEY was not set.


In [2]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
def merge_eupmc(source_path, output_path, columns):
    print("Starting merging search results from Europe PMC...")
    # process eupmc search results
    df = pd.read_csv(source_path, sep = ",")
    df = df[["DOI", "EXTERNAL_ID", "PMCID", "TITLE"]]
    df = df.rename(columns={"EXTERNAL_ID": "PMID", "TITLE": "Title"}, errors = "raise")
    for ind in df.index:
        print(ind)
        if(ind%10 == 0):
            time.sleep(random.randint(3,6)*10)
        proxies = plib.get_proxies()
        pmid = str(df["PMID"][ind])
        # print(pmid)
        url = "https://pubmed.ncbi.nlm.nih.gov/" + pmid + "/"
        regex = "[a-zA-Z]"
        if len(re.findall(regex, pmid)) == 0:
            # print("pmid")
            soup = plib.request_webpage(url)
            # print(soup)

            # get PMCID
            # print(df["PMCID"][ind])
            if df["PMCID"][ind] is np.nan:
                try:
                    pmcid = soup.find_all("span", {"class": "identifier pmc"})[0].find_all("a", {"class": "id-link"})[0].get_text().strip()
                except:
                    pmcid  ="not found"
            else:
                pmcid = str(df["PMCID"][ind])
            # print(pmcid)
            # get DOI
            if df["DOI"][ind] is np.nan:
                try:
                    doi = soup.find_all("span", {"class": "identifier doi"})[0].find_all("a", {"class": "id-link"})[0].get_text().strip()
                except:
                    doi  ="not found"
            else:
                doi = str(df["DOI"][ind])
            # get full_text_url
            if pmcid != "not found":
                full_text_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/" + pmcid + "/"
                full_text_source = "PMC"
            else:
                try:
                    full_text_url = soup.find_all("div", {"class": "full-text-links-list"})[0].find_all("a", {"class": "link-item dialog-focus"})[0]["href"].strip()
                    full_text_source = soup.find_all("div", {"class": "full-text-links-list"})[0].find_all("a", {"class": "link-item dialog-focus"})[0]["data-ga-action"].strip()
                except:
                    full_text_url = "not found"
                    full_text_source = "not found"
            # columns = ["DOI", "PMID", "PMCID", "Title", "First_Author", "full_text_url", "full_text_source"]
        else:
            # print("not pmid")
            if df["DOI"][ind] is np.nan:
                doi  ="not found"
            else:
                doi = str(df["DOI"][ind])
            if df["PMCID"][ind] is np.nan:
                full_text_url = "not found"
                full_text_source = "not found"
                pmcid = "not found"
            else:
                full_text_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/" + pmcid + "/"
                full_text_source = "PMC"
                pmcid = str(df["PMCID"][ind])
            first_author = "not found"
            pmid = "not found"
        row = {
            "DOI": [doi],
            "PMID": [pmid],
            "PMCID": [pmcid],
            "Title": [str(df["Title"][ind])],
            "First_Author": [first_author],
            "full_text_url": [full_text_url],
            "full_text_source": [full_text_source]
        }
        # print(row)
        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")
# --------------------start of test code--------------------
# source_path = fpath.poten_litera_eupmc
# output_path = fpath.poten_litera_eupmc_processed
# plib.clear_file(output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(source_path, sep=',')
# df = df[["SOURCE", "DOI", "EXTERNAL_ID", "PMCID", "TITLE"]]
# print(df.head(3))
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# col_one_list = set(df['SOURCE'].tolist())
# print(col_one_list)
# 
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# print(df["SOURCE"].isnull().values.any())
# print(df["DOI"].isnull().values.any())
# print(df["EXTERNAL_ID"].isnull().values.any())
# print(df["PMCID"].isnull().values.any())
# print(df["TITLE"].isnull().values.any())
# # PMID, Title don't contain np.nan
# # DOI, PMCID contain np.nan
# # we need to fill in what are missing
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# merge_eupmc(source_path, output_path, columns)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# ---------------------end of test code---------------------

In [4]:
# get pmid from doi
def doi2pmid(doi):
    pmid = ""
    if pmid == None:
        return np.nan
    else:
        return pmid
# --------------------start of test code--------------------
doi = "10.1007/BF00231845"
# pmid = plib.doi2pmid(doi)
# print(pmid)
pmid = plib.id_converter_in_pubmed(doi, "pmid")
# ---------------------end of test code---------------------

FeatureNotFound: Couldn't find a tree builder with the features you requested: json. Do you need to install a parser library?

In [ ]:
def merge_google_shcolar(source_path, output_path, columns):
    print("Starting merging search results from Google Scholar...")
    df = pd.read_csv(source_path, header=None, sep=',')
    df.columns = ["title", "url", "full_text_url", "full_text_type", "full_text_source"]
    for ind in df.index:
        # columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
        # get title
        title = str(df["title"][ind]).strip()
        # get pmid
        pmid = np.nan
        # get pmcid
        pmcid = np.nan

        # get full_text_url
        # get full_text_source
        if str(df["full_text_type"][ind]) == "[HTML]":
            full_text_url, = plib.get_final_redirected_url(str(df["full_text_url"][ind]).strip()).strip()
            full_text_source = full_text_url.split("://")[1].strip().split("/")[0].strip()
        elif str(df["full_text_type"][ind]) == "not found" | "[PDF]" | "UB":
            if str(df["url"][ind]) != "not found":
                full_text_url, = plib.get_final_redirected_url(str(df["url"][ind]).strip()).strip()
                full_text_source = full_text_url.split("://")[1].strip().split("/")[0].strip()
            else:
                full_text_url = np.nan
                full_text_source = np.nan
        else:
            raise Exception("Found a full_text_type not expected!")
        # get pdf_url
        # get pdf_source
        if str(df["full_text_type"][ind]) == "[PDF]":
            pdf_url = str(df["full_text_url"][ind])
            pdf_source = pdf_url.split("://")[1].strip().split("/")[0].strip()
        else:
            pdf_url = np.nan
            pdf_source = np.nan

        # get DOI
        proxies = plib.get_proxies()
        url = full_text_url
        doi = get_doi(url)
        
        # columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
        row = {
            "DOI": [doi],
            "PMID": [pmid],
            "PMCID": [pmcid],
            "Title": [str(df["Title"][ind])],
            "full_text_url": [full_text_url],
            "full_text_source": [full_text_source],
            "pdf_url": [pdf_url],
            "pdf_source": [pdf_source]
        }
        # print(row)
        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")
# --------------------start of test code--------------------
# source_path = fpath.poten_litera_gs
# output_path = fpath.poten_litera_gs_processed
# plib.clear_file(output_path)

# --------------------start of test code--------------------
# df = pd.read_csv(source_path, header=None, sep=',')
# df.columns = ["title", "url", "full_text_url", "full_text_type", "full_text_source"]
# print(df.head(3))
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# full_text_type = set(df['full_text_type'].tolist())
# print("All full_text_type include: ",full_text_type)
# full_text_source = set(df['full_text_source'].tolist())
# print("All full_text_source include: ", full_text_source)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# # ["title", "url", "full_text_url", "full_text_type", "full_text_source"]
# print(df["title"].str.contains('not found').sum())
# print(df["url"].str.contains('not found').sum())
# print(df["full_text_url"].str.contains('not found').sum())
# print(df["full_text_type"].str.contains('not found').sum())
# print(df["full_text_source"].str.contains('not found').sum())
# # 
# # title, url don't contain np.nan
# # full_text_url, full_text_type, full_text_source contain np.nan
# # we need to fill in what are missing
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# merge_google_shcolar(source_path, output_path, columns)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# ---------------------end of test code---------------------

In [ ]:
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException

In [ ]:
    # options = Options()
    # options.add_argument('--headless')
    # driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))
    # # driver = webdriver.Firefox(GeckoDriverManager().install())
    # driver.get("https://www.pmid2cite.com/doi-to-pmid-converter")
    # WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, "//p[text()='Consent']"))).click()
    # doi = "10.1113/JP282626"
    # try:
    #     # print(str(doi))
    #     WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#formInput"))).send_keys(str(doi).strip())
    # except TimeoutException:
    #     print("Waiting for clicking consent timeout")
    # try:
    #     # driver.find_element(By.XPATH, "/html/body/div[5]/div[2]/form/button").click()
    #     # EC.presence_of_element_located(By.XPATH, "/html/body/div[5]/div[2]/form/button")
    #     WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[2]/form/button"))).click()
    # except TimeoutException:
    #     print("Waiting for clicking button timeout")
    # try:
    #     WebDriverWait(driver, 30).until(EC.visibility_of_all_elements_located((By.XPATH, "/html/body/div[7]/div[3]/p[1]/span[2]/a")))
    # except TimeoutException:
    #     print("Waiting for getting PMID timeout")
    #     my_elem = driver.find_element(By.XPATH, "button[aria-label='Leaving from']").text
    #     res = my_elem.text
    #     pmid = res

    #     pmid = np.nan
    # print(pmid)
    # driver.quit()